In [56]:
###############################
# load previous save model
###############################

import numpy as np
import pandas as pd
import gc
import csv

import warnings
warnings.filterwarnings('ignore')

import os
import datetime
import time
import glob
import os.path as osp
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torchvision import datasets, models
import torchvision.transforms as transforms
from torch.utils import data as D
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from tensorboardX import SummaryWriter

class DefaultConfig(object):
    path = '../data/compcars/data/cropped_image/'   #../data/compcars/data/image
    labelPath = '../data/compcars/data/label/'
    miscPath = '../data/compcars/data/misc/'
    modleNameFile = '../data/compcars/data/misc/modle_names.csv'
    makeNameFile = '../data/compcars/data/misc/make_names.csv'
    trainFilename = '../data/compcars/data/train_test_split/classification/train.txt'
    testFilename = '../data/compcars/data/train_test_split/classification/test.txt'
    verificationFilename = '../data/compcars/data/train_test_split/verification/verification_train.txt'
    log_dir = '../log/'
    num_workers = 8
    num_epoches = 100
    lr = 0.1
    lr_decay = 0.95
    max_epoch = 200000
    weight_decay = 1e-4
    load_model_path = '../checkpoints/train.pth'
    previous_loss = 1e100
    modle_csv_file = 'modle_names.csv'

opt = DefaultConfig()

# DataSet and DataLoader
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)


class CompcarsDS(D.Dataset):
    """
    A customized data loader.
    """
    def __init__(self, root, mode='train'):
        """ Intialize the dataset
        """
        self.filenames = []
        self.root = root
        self.transform = transforms.Compose([
            transforms.RandomRotation(10),
            transforms.Scale(299),
            transforms.CenterCrop(299), 
            transforms.ToTensor(),
            normalize
        ])
                    
        # below for train dataset selected by the paper
        with open(opt.trainFilename, newline='\n') as trainfile:
            for line in trainfile:
                self.filenames.append(opt.path + line.replace("\n",""))
        
        self.len = len(self.filenames)
        
    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        image = Image.open(self.filenames[index])
        labelStr = self.filenames[index].split("/")[6]
        label = paperTrainFeatureList.index(labelStr)
        return self.transform(image), label
    
    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

 
"""
Verification dataset
"""
verificationCars = CompcarsDS(opt.path)
print(verificationCars.len)   
    
####   verification dataset Loader
##########################################################
verificationloader = D.DataLoader(verificationCars, batch_size=1, shuffle=True, num_workers=8, pin_memory=True)

    

# Load the model 
continue_model_inception3 = torchvision.models.inception_v3(pretrained=None)
net = continue_model_inception3

# # # modify output classes
num_ftrs = net.fc.in_features
num_classes = 431
net.fc = torch.nn.Linear(num_ftrs, num_classes)    #num_ftrs = 2048

criterion = torch.nn.CrossEntropyLoss()
criterion.cuda()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

checkpoint = torch.load(opt.load_model_path)
net.load_state_dict(checkpoint['model_state_dict'])  

if torch.cuda.device_count() > 1:
    net = torch.nn.DataParallel(net)
net.cuda()


##############################
# calculate the features size from paper's training dataset
##############################
paperTrainFeatureSet = set()
with open(opt.trainFilename, newline='\n') as trainfile:
            for line in trainfile:
                feature = line.split('/')[1]
                paperTrainFeatureSet.add(feature)
print('total train feature size: %s' %(len(paperTrainFeatureSet)))
num_classes = len(paperTrainFeatureSet)
paperTrainFeatureList = sorted(paperTrainFeatureSet)
featureIndex = paperTrainFeatureList.index('104')
print(featureIndex)

compcarsFeatures = {}
modle_csv = os.path.join(opt.miscPath, opt.modle_csv_file)
with open(modle_csv, newline='\n') as modlefile:
    for line in modlefile:
        items = line.split(',')
        key, values = items[0], items[1]
        compcarsFeatures[key] = values

print('total compcars dataset feature size:', len(compcarsFeatures))



16016
total train feature size: 431
12
total compcars dataset feature size: 2004


In [57]:
featureList = []
inFC =  torch.zeros(1, 2048).cuda()
outFC = torch.zeros(1, 431).cuda()

inFC1 = []
outFC1 = []


def hook2(module, i, o):
    inFC1.append(i)
    outFC1.append(o)
    
model = net  
dataloders = verificationloader
model.eval()
model.module.fc.register_forward_hook(hook2)
featureList = []
currentFeature = []

for i, (inputs, labels) in enumerate(dataloders):
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
    outputs = model(inputs)
    _, preds = torch.max(outputs.data, 1)
    i += 1
    
    in2048 = inFC1[0][0][0].data
    out431 = outFC1[0][0].data
    currentFeature = [in2048, out431, labels.item(), preds.item()]
    featureList.append(currentFeature)

    currentFeature = []
    inFC1 = []
    outFC1 = []
  
    
print(len(featureList))
feature_extract_tensor_file = '../checkpoints/feature_extract_full.pth'
torch.save(featureList, feature_extract_tensor_file)
        
